In [2]:
from project import data_frames

for key, values in data_frames.items():
    print(key)

energy-data/power-laws-forecasting-energy-consumption-holidays.csv
energy-data/power-laws-forecasting-energy-consumption-metadata.csv
energy-data/power-laws-forecasting-energy-consumption-submission-forecast-period.csv
energy-data/power-laws-forecasting-energy-consumption-test-data.csv
energy-data/power-laws-forecasting-energy-consumption-training-data.csv
energy-data/power-laws-forecasting-energy-consumption-weather.csv


# Proyecto Minería


# Clustering

In [36]:
!pip install pyspark
!pip install -U -q PyDrive
!sudo apt-get install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Selecting previously unselected package java-common.
(Reading database ... 68308 files and directories currently installed.)
Preparing to unpack .../0-java-common_0.72_all.deb ...
Unpacking java-common (0.72) ...
Selecting previously unselected package libpcsclite1:amd64.
Preparing to unpack .../1-libpcsclite1_1.8.26-3_amd64.deb ...
Unpacking libpcsclite1:amd64 (1.8.26-3) ...
Selecting previously unselected package libxtst6:amd64.
Preparing to unpack .../2-libxtst6_2%3a1.2.3-1_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../3-openjdk-8-jre-headless_8u382-ga-1~20.04.1_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u382-ga-1~20.04.1) ...
Selecting previously unselected package ca-certificates-java.
Preparing to unpack .../4-ca-certificates-java_20190405ubuntu1.1_all.deb ...
Unpacking ca-certificates-java (20190405ubuntu1.1) ...
Selecting previously unselected package openjdk-8-jdk

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [38]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

23/09/30 18:05:38 WARN Utils: Your hostname, codespaces-f5d2da resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
23/09/30 18:05:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/30 18:05:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [41]:
holidays = data_frames['energy-data/power-laws-forecasting-energy-consumption-holidays.csv']
metadata = data_frames['energy-data/power-laws-forecasting-energy-consumption-metadata.csv']
forecast = data_frames['energy-data/power-laws-forecasting-energy-consumption-submission-forecast-period.csv']
test_data = data_frames['energy-data/power-laws-forecasting-energy-consumption-test-data.csv']
train_data = data_frames['energy-data/power-laws-forecasting-energy-consumption-training-data.csv']
weather = data_frames['energy-data/power-laws-forecasting-energy-consumption-weather.csv']

In [43]:
total_data = pd.concat([train_data, test_data])

In [44]:
# Convert the Timestamp column to datetime
total_data['Timestamp'] = pd.to_datetime(total_data['Timestamp'])

# Group by Timestamp and SiteId, and calculate the mean value
result_total_data = total_data.groupby(['SiteId', pd.Grouper(key='Timestamp', freq='H')])['Value'].mean().reset_index()

result_total_data

,SiteId,Timestamp,Value
0,1,2014-09-03 00:00:00+00:00,9.096555e+05
1,1,2014-09-04 00:00:00+00:00,1.748273e+06
2,1,2014-09-05 00:00:00+00:00,NaN
3,1,2014-09-06 00:00:00+00:00,NaN
4,1,2014-09-07 00:00:00+00:00,NaN
...,...,...,...
3149111,305,2015-11-21 08:00:00+00:00,2.146361e+03
3149112,305,2015-11-21 09:00:00+00:00,4.210141e+03
3149113,305,2015-11-21 10:00:00+00:00,3.944624e+03
3149114,305,2015-11-21 11:00:00+00:00,3.981655e+03


### Se organizan los datos en orden cronologico

In [45]:
data_spark = spark.createDataFrame(result_total_data)

/home/codespace/.python/current/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:
/home/codespace/.python/current/lib/python3.10/site-packages/pyspark/sql/pandas/types.py:379: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(s.dtype):


In [46]:
data_ordenado=data_spark.sort('Timestamp')

In [47]:
data_spark.show()

23/09/30 18:18:00 WARN TaskSetManager: Stage 0 contains a task of very large size (16946 KiB). The maximum recommended task size is 1000 KiB.
23/09/30 18:18:05 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 0 (TID 0): Attempting to kill Python Worker


+------+-------------------+------------------+
|SiteId|          Timestamp|             Value|
+------+-------------------+------------------+
|     1|2014-09-03 00:00:00|  909655.510515214|
|     1|2014-09-04 00:00:00| 1748273.124752296|
|     1|2014-09-05 00:00:00|               NaN|
|     1|2014-09-06 00:00:00|               NaN|
|     1|2014-09-07 00:00:00|               NaN|
|     1|2014-09-08 00:00:00|  1964877.99902387|
|     1|2014-09-09 00:00:00|  3266903.91769804|
|     1|2014-09-10 00:00:00|2926094.4745757496|
|     1|2014-09-11 00:00:00|1720502.1513768754|
|     1|2014-09-12 00:00:00|1136248.1075743749|
|     1|2014-09-13 00:00:00|1040528.6001987156|
|     1|2014-09-14 00:00:00|1030667.6855087331|
|     1|2014-09-15 00:00:00|1226753.4244866522|
|     1|2014-09-16 00:00:00|1228874.2899190905|
|     1|2014-09-17 00:00:00|  1169026.41570396|
|     1|2014-09-18 00:00:00|1513721.7344672228|
|     1|2014-09-19 00:00:00|2015528.3234096123|
|     1|2014-09-20 00:00:00|2548942.6232

In [48]:
data_ordenado.show()

23/09/30 18:18:09 WARN TaskSetManager: Stage 1 contains a task of very large size (16946 KiB). The maximum recommended task size is 1000 KiB.


+------+-------------------+-----------------+
|SiteId|          Timestamp|            Value|
+------+-------------------+-----------------+
|   297|2009-06-23 06:00:00| 96408.8267305633|
|   297|2009-06-23 07:00:00|76800.25180231314|
|   297|2009-06-23 08:00:00|75166.20389162563|
|   297|2009-06-23 09:00:00| 71898.1080702506|
|   297|2009-06-23 10:00:00|73532.15598093811|
|   297|2009-06-23 11:00:00| 71898.1080702506|
|   297|2009-06-23 12:00:00| 71898.1080702506|
|   297|2009-06-23 13:00:00|70264.06015956309|
|   297|2009-06-23 14:00:00| 71898.1080702506|
|   297|2009-06-23 15:00:00| 71898.1080702506|
|   297|2009-06-23 16:00:00|70264.06015956309|
|   297|2009-06-23 17:00:00| 71898.1080702506|
|   297|2009-06-23 18:00:00| 71898.1080702506|
|   297|2009-06-23 19:00:00| 71898.1080702506|
|   297|2009-06-23 20:00:00|78434.29971300067|
|   297|2009-06-23 21:00:00|78434.29971300067|
|   297|2009-06-23 22:00:00|80068.34762368817|
|   297|2009-06-23 23:00:00|81702.39553437567|
|   297|2009-

In [49]:
# Convert the Timestamp column to datetime
weather['Timestamp'] = pd.to_datetime(weather['Timestamp'])

# Group by Timestamp and SiteId, and calculate the mean temperature
result = weather.groupby(['SiteId', pd.Grouper(key='Timestamp', freq='H')])['Temperature'].median().reset_index()

result

,SiteId,Timestamp,Temperature
0,2,2013-01-01 13:00:00+00:00,21.85
1,2,2013-01-01 14:00:00+00:00,22.00
2,2,2013-01-01 15:00:00+00:00,22.00
3,2,2013-01-01 16:00:00+00:00,22.70
4,2,2013-01-01 17:00:00+00:00,22.00
...,...,...,...
1284292,57,2017-12-30 18:00:00+00:00,8.00
1284293,57,2017-12-30 19:00:00+00:00,8.70
1284294,57,2017-12-30 20:00:00+00:00,8.00
1284295,57,2017-12-30 21:00:00+00:00,7.00


In [50]:
data_weatherpark = spark.createDataFrame(result)

/home/codespace/.python/current/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:
/home/codespace/.python/current/lib/python3.10/site-packages/pyspark/sql/pandas/types.py:379: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(s.dtype):


In [51]:
data_weatherorden=data_weatherpark.sort('Timestamp')

In [52]:
data_weatherpark.show()

+------+-------------------+-----------+
|SiteId|          Timestamp|Temperature|
+------+-------------------+-----------+
|     2|2013-01-01 13:00:00|      21.85|
|     2|2013-01-01 14:00:00|       22.0|
|     2|2013-01-01 15:00:00|       22.0|
|     2|2013-01-01 16:00:00|       22.7|
|     2|2013-01-01 17:00:00|       22.0|
|     2|2013-01-01 18:00:00|       22.0|
|     2|2013-01-01 19:00:00|      20.75|
|     2|2013-01-01 20:00:00|       20.0|
|     2|2013-01-01 21:00:00|       17.0|
|     2|2013-01-01 22:00:00|       16.0|
|     2|2013-01-01 23:00:00|       16.0|
|     2|2013-01-02 00:00:00|       15.0|
|     2|2013-01-02 01:00:00|       15.0|
|     2|2013-01-02 02:00:00|       15.0|
|     2|2013-01-02 03:00:00|       15.0|
|     2|2013-01-02 04:00:00|      13.95|
|     2|2013-01-02 05:00:00|       15.0|
|     2|2013-01-02 06:00:00|       15.0|
|     2|2013-01-02 07:00:00|      15.95|
|     2|2013-01-02 08:00:00|       17.0|
+------+-------------------+-----------+
only showing top

23/09/30 18:20:56 WARN TaskSetManager: Stage 2 contains a task of very large size (6901 KiB). The maximum recommended task size is 1000 KiB.


In [53]:
data_weatherorden.show()

23/09/30 18:21:00 WARN TaskSetManager: Stage 3 contains a task of very large size (6901 KiB). The maximum recommended task size is 1000 KiB.


+------+-------------------+-----------+
|SiteId|          Timestamp|Temperature|
+------+-------------------+-----------+
|     3|2013-01-01 13:00:00|      21.85|
|    10|2013-01-01 13:00:00|      25.85|
|     4|2013-01-01 13:00:00|      25.85|
|     2|2013-01-01 13:00:00|      21.85|
|     5|2013-01-01 13:00:00|      21.85|
|     6|2013-01-01 13:00:00|      21.85|
|     8|2013-01-01 13:00:00|      25.85|
|     9|2013-01-01 13:00:00|      25.85|
|    10|2013-01-01 14:00:00|       27.0|
|     4|2013-01-01 14:00:00|       27.0|
|     2|2013-01-01 14:00:00|       22.0|
|     5|2013-01-01 14:00:00|       22.0|
|     3|2013-01-01 14:00:00|       22.0|
|     6|2013-01-01 14:00:00|       22.0|
|     8|2013-01-01 14:00:00|       27.0|
|     9|2013-01-01 14:00:00|       27.0|
|    10|2013-01-01 15:00:00|       27.0|
|     2|2013-01-01 15:00:00|       22.0|
|     4|2013-01-01 15:00:00|       27.0|
|     3|2013-01-01 15:00:00|       22.0|
+------+-------------------+-----------+
only showing top

In [54]:
holidayspark = spark.createDataFrame(holidays)

In [55]:
holidayspark.show()

+----------+--------------------+------+
|      Date|             Holiday|SiteId|
+----------+--------------------+------+
|2016-02-15|Washington's Birt...|     1|
|2017-05-29|        Memorial Day|     1|
|2017-11-23|    Thanksgiving Day|     1|
|2017-12-29| New Years Eve Shift|     1|
|2017-12-31|       New Years Eve|     1|
|2014-01-01|            New year|     1|
|2015-01-19|Birthday of Marti...|     1|
|2017-01-02| New year (Observed)|    12|
|2017-11-10|Veterans Day (Obs...|    12|
|2017-01-01|            New year|    13|
|2015-05-25|        Memorial Day|    13|
|2016-11-11|        Veterans Day|    14|
|2016-11-24|    Thanksgiving Day|    14|
|2016-12-26|Christmas Day (Ob...|    14|
|2017-07-04|    Independence Day|    14|
|2017-09-04|           Labor Day|    14|
|2017-12-26|Washington's Birt...|    14|
|2015-01-19|Birthday of Marti...|    14|
|2015-07-04|    Independence Day|    14|
|2015-09-07|           Labor Day|    14|
+----------+--------------------+------+
only showing top

In [56]:
metadataspark = spark.createDataFrame(metadata)

In [57]:
metadataspark.show()

+------+------------------+--------+---------------+--------------+---------------+-----------------+----------------+--------------+----------------+--------------+
|SiteId|           Surface|Sampling|BaseTemperature|MondayIsDayOff|TuesdayIsDayOff|WednesdayIsDayOff|ThursdayIsDayOff|FridayIsDayOff|SaturdayIsDayOff|SundayIsDayOff|
+------+------------------+--------+---------------+--------------+---------------+-----------------+----------------+--------------+----------------+--------------+
|   207| 7964.873346500099|    30.0|           18.0|         false|          false|            false|           false|         false|            true|          true|
|     7| 15168.12597088516|    30.0|           18.0|         false|          false|            false|           false|         false|            true|          true|
|    74| 424.3406632660189|    15.0|           18.0|         false|          false|            false|           false|         false|            true|          true|
|   

In [58]:
temp1 = total_data.merge(weather, how = 'inner', left_on= ['Timestamp','SiteId'], right_on=['Timestamp', 'SiteId'])

temp1.shape

(502215, 7)

In [59]:
temp1=data_ordenado.join(data_weatherorden, ['Timestamp','SiteId'], 'inner')

In [60]:
temp1.count()

23/09/30 18:21:24 WARN TaskSetManager: Stage 6 contains a task of very large size (16946 KiB). The maximum recommended task size is 1000 KiB.
23/09/30 18:21:26 WARN TaskSetManager: Stage 7 contains a task of very large size (6901 KiB). The maximum recommended task size is 1000 KiB.


461657

In [61]:
len(temp1.columns)

4

In [62]:
temp1.show()

23/09/30 18:21:35 WARN TaskSetManager: Stage 15 contains a task of very large size (16946 KiB). The maximum recommended task size is 1000 KiB.
23/09/30 18:21:37 WARN TaskSetManager: Stage 16 contains a task of very large size (6901 KiB). The maximum recommended task size is 1000 KiB.


+-------------------+------+------------------+-----------+
|          Timestamp|SiteId|             Value|Temperature|
+-------------------+------+------------------+-----------+
|2013-01-01 13:00:00|     2| 61857.48899385976|      21.85|
|2013-01-01 14:00:00|     5|24611.560309614106|       22.0|
|2013-01-01 15:00:00|     5|24762.356712672045|       22.0|
|2013-01-01 17:00:00|     6|24442.445493296873|       22.0|
|2013-01-01 18:00:00|     5|23730.591849644043|       22.0|
|2013-01-01 20:00:00|     5|22936.926570391734|       20.0|
|2013-01-01 21:00:00|     6| 25884.32394101801|       17.0|
|2013-01-01 22:00:00|     6|25646.906204646257|       16.0|
|2013-01-01 23:00:00|     5|24000.438044589828|       16.0|
|2013-01-02 00:00:00|     3|320703.53464475373|       15.0|
|2013-01-02 00:00:00|     6|25177.861408399614|       15.0|
|2013-01-02 03:00:00|     5| 19976.55507878062|       15.0|
|2013-01-02 05:00:00|     2|  35658.7548051865|       15.0|
|2013-01-02 10:00:00|     5| 24286.15754

In [63]:
temp1.show()

23/09/30 18:21:44 WARN TaskSetManager: Stage 20 contains a task of very large size (16946 KiB). The maximum recommended task size is 1000 KiB.
23/09/30 18:21:46 WARN TaskSetManager: Stage 21 contains a task of very large size (6901 KiB). The maximum recommended task size is 1000 KiB.


+-------------------+------+------------------+-----------+
|          Timestamp|SiteId|             Value|Temperature|
+-------------------+------+------------------+-----------+
|2013-01-01 13:00:00|     2| 61857.48899385976|      21.85|
|2013-01-01 14:00:00|     5|24611.560309614106|       22.0|
|2013-01-01 15:00:00|     5|24762.356712672045|       22.0|
|2013-01-01 17:00:00|     6|24442.445493296873|       22.0|
|2013-01-01 18:00:00|     5|23730.591849644043|       22.0|
|2013-01-01 20:00:00|     5|22936.926570391734|       20.0|
|2013-01-01 21:00:00|     6| 25884.32394101801|       17.0|
|2013-01-01 22:00:00|     6|25646.906204646257|       16.0|
|2013-01-01 23:00:00|     5|24000.438044589828|       16.0|
|2013-01-02 00:00:00|     3|320703.53464475373|       15.0|
|2013-01-02 00:00:00|     6|25177.861408399614|       15.0|
|2013-01-02 03:00:00|     5| 19976.55507878062|       15.0|
|2013-01-02 05:00:00|     2|  35658.7548051865|       15.0|
|2013-01-02 10:00:00|     5| 24286.15754

In [64]:
temp2 = temp1.join(metadataspark, 'SiteId','inner')

In [65]:
temp2.show()

23/09/30 18:21:54 WARN TaskSetManager: Stage 25 contains a task of very large size (16946 KiB). The maximum recommended task size is 1000 KiB.
23/09/30 18:21:55 WARN TaskSetManager: Stage 26 contains a task of very large size (6901 KiB). The maximum recommended task size is 1000 KiB.


+------+-------------------+------------------+-----------+------------------+--------+---------------+--------------+---------------+-----------------+----------------+--------------+----------------+--------------+
|SiteId|          Timestamp|             Value|Temperature|           Surface|Sampling|BaseTemperature|MondayIsDayOff|TuesdayIsDayOff|WednesdayIsDayOff|ThursdayIsDayOff|FridayIsDayOff|SaturdayIsDayOff|SundayIsDayOff|
+------+-------------------+------------------+-----------+------------------+--------+---------------+--------------+---------------+-----------------+----------------+--------------+----------------+--------------+
|    26|2014-11-27 14:00:00|367090.93201896665|       11.0|14.884534437442198|    15.0|           18.0|         false|          false|            false|           false|         false|            true|          true|
|    26|2014-11-27 17:00:00| 380214.5218407182|       9.95|14.884534437442198|    15.0|           18.0|         false|          fals

In [66]:
temp2.count()

23/09/30 18:22:02 WARN TaskSetManager: Stage 37 contains a task of very large size (16946 KiB). The maximum recommended task size is 1000 KiB.
23/09/30 18:22:04 WARN TaskSetManager: Stage 38 contains a task of very large size (6901 KiB). The maximum recommended task size is 1000 KiB.


461657

In [67]:
energy_data = temp2[['SiteId', 'Value', 'Temperature', 'Surface']]
energy_data.show()

23/09/30 18:22:09 WARN TaskSetManager: Stage 54 contains a task of very large size (16946 KiB). The maximum recommended task size is 1000 KiB.
23/09/30 18:22:10 WARN TaskSetManager: Stage 55 contains a task of very large size (6901 KiB). The maximum recommended task size is 1000 KiB.


+------+------------------+-----------+------------------+
|SiteId|             Value|Temperature|           Surface|
+------+------------------+-----------+------------------+
|    26|367090.93201896665|       11.0|14.884534437442198|
|    26| 380214.5218407182|       9.95|14.884534437442198|
|    26| 369513.9853800378|       9.35|14.884534437442198|
|    26|356623.18967688765|        9.0|14.884534437442198|
|    26| 440533.8594959018|       13.0|14.884534437442198|
|    26|  423050.172770459|       12.0|14.884534437442198|
|    26| 422010.9196915024|      11.75|14.884534437442198|
|    26|187789.09143120135|       11.9|14.884534437442198|
|    26|205557.23649427813|       10.2|14.884534437442198|
|    26| 501100.9055731932|      15.25|14.884534437442198|
|    26| 416228.3376006938|       16.9|14.884534437442198|
|    26|286919.43586505123|       12.7|14.884534437442198|
|    26|177929.62047344298|        9.9|14.884534437442198|
|    26|371317.08299725526|       11.0|14.88453443744219

In [ ]:
df_categorizado = df.withColumn(
    "Categoria",
    when(df["Edad"] < 30, "Joven")
    .when((df["Edad"] >= 30) & (df["Edad"] < 40), "Adulto")
    .when((df["Edad"] >= 40) & (df["Edad"] < 50), "Mediana Edad")
    .when(df["Edad"] >= 50, "Mayor")
)

In [ ]:
energy_data_categoria=energy_data.withColumn("Categoria_surface",
                                             when(energy_data["Surface"]<(max('Surface')-min('surface')/3)))